In [979]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [980]:
#function to make sure the date is a valid date for sorting data and not one from the future
#this function will add the year onto the date as it currently doesn't have it 
def adjust_date(row):
    
    season_year = row['Season']
    month = row['Date'].split(' ')[0] 
    day = row['Date'].split(' ')[1]
    
    month_numbers = {
        "January": 1, "February": 2, "March": 3, "April": 4,
        "May": 5, "June": 6, "July": 7, "August": 8,
        "September": 9, "October": 10, "November": 11, "December": 12
    }
    month_number = month_numbers[month]

    #adjusting for january and february games as the season rolls into the next year
    year = season_year + 1 if month_number in [1, 2] else season_year

    # Return the adjusted date in MM/DD/YYYY format
    return f"{month_number:02d}/{day}/{year}"

In [981]:
#cleaning data to prepare it for machine learning
#matches.dtypes
matches = pd.read_csv('matches.csv')

#Removing playoff row error due to differing season lengths
#playoffIdx = match_df[ (match_df['Date'] == 'Playoffs')].index
#match_df =  match_df.drop(playoffIdx)
#match_df = match_df.reset_index(drop = 'true')
#match_df.to_csv('matches.csv', index=False, encoding='utf-8')
#match_df


#Removing playoff row error due to differing season lengths
playoffIdx = matches[ (matches['Date'] == 'Playoffs')].index
matches =  matches.drop(playoffIdx)
matches = matches.reset_index(drop = 'true')
matches.to_csv('matches.csv', index=False, encoding='utf-8')

#converting dates to proper dates so that they can be stored as date types
matches['Date'] = matches.apply(adjust_date, axis=1)
matches['Date'] = pd.to_datetime(matches['Date'])


#converting home/away into numeric data
matches['Home/AwayCode'] = matches['Home/Away'].astype('category').cat.codes #0 is away, 1 is home



#converting team into numeric data
#matches['TeamCode'] = matches['Team'].astype('category').cat.codes

#converting opponents into numeric data
#matches['OppCode'] = matches['Opp'].astype('category').cat.codes

#adding hour column 
matches['Hour'] = matches['Time'].str.replace(':.+', '', regex = True).astype('int')


#adding day of the week column
matches['DayCode'] = matches['Date'].dt.dayofweek
#NOTE STILL NEED TO CLEAN UP DATE FIRST


#converting all other stats from objects to ints
for column in matches.columns[10:24]:
    matches[column] = pd.to_numeric(matches[column])

#adding in team year and opp year column to make teams separate by year
matches['TeamYear'] = matches['Team'].astype('str') + '' + matches['Season'].apply(str)
matches['OppYear'] = matches['Opp'].astype('str') + '' + matches['Season'].apply(str)


matches['TeamCode'] = matches['TeamYear'].astype('category').cat.codes

matches['OppCode'] = matches['OppYear'].astype('category').cat.codes

#converting Win/Loss to numbers
matches['Target'] = (matches['Result'] == 'W').astype('int')

In [1216]:
#Initializing a random forest machine learning model
rf = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, random_state = 1)

#Training on previous games from this and the past two seasons
training = matches[matches['Date'] < '12-24-2024']
train = matches[matches['Date'] > '04-04-2022'] #delete this and change above line to fix
test = matches[matches['Date'] > '12-24-2024']

#Testing using 3 basic predictors, whether the game is home or away,
#who the team playing is and who the opponent is
#team's and oppoenents are broken down by year so the 2023 49ers are different than the 2024 49ers
#this helps to account for the NFL's volatility in team strength season to season
predictors = ['Home/AwayCode', 'TeamCode', 'OppCode']
rf.fit(train[predictors], train['Target'])
preds = rf.predict(test[predictors])

In [1215]:
acc = accuracy_score(test['Target'], preds)
acc

0.78125

In [1210]:
precision_score(test['Target'],preds)

0.2222222222222222

In [1211]:
def mapWin(col):
    col.apply(lambda x : 'W' if x == 1 else 'L')

In [985]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('Date')
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [1218]:
def make_predictions(data, predVals):
    rf = RandomForestClassifier(n_estimators = 100, min_samples_split = 15, random_state = 1)
    training = data[data['Date'] < '04-04-2024']
    train = training[training['Date'] > '04-04-2022'] #cuts off the data from only the past two seasons as it tends to make the model more accurate surprisingly
    test = data[data['Date'] > '04-04-2024']
    rf.fit(train[predVals], train['Target'])
    preds = rf.predict(test[predVals])
    combined = pd.DataFrame(dict(actual=test['Target'], prediction=preds))
    precision = precision_score(test['Target'],preds)
    accuracy = accuracy_score(test['Target'], preds)
    return combined, precision, accuracy, rf

In [1281]:

#Used to bypass deprecation warning but still have the 'Team' index included in the dataframe
cols = ['TmSc', 'OppSc', 'Off1stD', 'OffTotYd', 'OffPassY',
       'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY',
       'DefTO', 'EPOff', 'EPDef', 'EPSp.']
new_cols = [f'{col}Rolling' for col in cols]
matches_rolling = matches.set_index('Team').groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols),include_groups = False)
matches_rolling = matches_rolling.droplevel(0)
matches_rolling = matches_rolling.reset_index()



predictorTests = []
predictorTests.append([])
predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 
                       'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.'])
predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO'])
predictorTests.append(['OffTotYd', 'DefTotYd', 'OffRushY', 'OffTO'])
predictorTests.append(['TmSc', 'OppSc']) #should always be 100% as TmSc > OppSc = win
predictorTests.append(['EPOff', 'EPDef', 'EPSp.'])
predictorTests.append(['Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])
predictorTests.append(['OffTotYdRolling', 'DefTotYdRolling', 'OffRushYRolling', 'OffTORolling'])

predictorTests.append(['Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])

predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 
                       'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])


#predictorTests.append(['Home/AwayCode', 'OppCode', 'TeamCode'])




rf_models = []
predictors = ['Home/AwayCode', 'TeamCode', 'OppCode']
for predictorCols in predictorTests:
    new_predictors = predictors + predictorCols
    combined, precision, accuracy, rf = make_predictions(matches_rolling, new_predictors)
    rf_models.append(rf)
    print("Predictors:", new_predictors)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")

    #This shows how many times a correct prediction was made when one team was predicted to win and the other was predicted to lose
    combined = combined.merge(matches_rolling[['Date','Team','Opp','Result']], left_index = True, right_index = True)
    merged = combined.merge(combined, left_on = ['Date', 'Team'], right_on = ['Date', 'Opp'])
    #Prints out the importance of the different features used to predict games
    feature_importances = rf.feature_importances_
    print("Feature Importances")
    for feature, importance in zip(new_predictors, feature_importances):
        print(f"{feature}: {importance}") 

    #remapping 0's and 1's to losses and wins
    mapping_cols = ['actual_x', 'actual_y', 'prediction_x', 'prediction_y']
    for col in mapping_cols:
        merged[col] = merged[col].apply(lambda x : 'W' if x == 1 else 'L')
        

    #prints correct win predictions
    print(merged[(merged['prediction_x'] == 'W') & (merged['prediction_y'] == 'L')]['actual_x'].value_counts())
    
    print("-"*200)
    print("")

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode']
Accuracy: 63.28%
Precision: 60.55%
Feature Importances
Home/AwayCode: 0.04307600625709231
TeamCode: 0.4763937883117916
OppCode: 0.48053020543111613
actual_x
W    150
L     94
Name: count, dtype: int64
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.']
Accuracy: 87.50%
Precision: 85.89%
Feature Importances
Home/AwayCode: 0.002053501226539676
TeamCode: 0.01621054675609453
OppCode: 0.013931551096698096
Off1stD: 0.023578192489112845
OffTotYd: 0.05638591020703362
OffPassY: 0.020328701086342518
OffRushY: 0.027141133942076706
OffTO: 0.06657606233904963
Def1stD: 0.03308844877844032
DefTotYd: 0.05996673426

In [1319]:
def joinGames(game_list):
    # Create a unique game identifier by sorting team names
    game_list['game_id'] = game_list.apply(lambda row: '_'.join(sorted([row['Team'], row['Opp']])), axis=1)
    
    # Keep only the first occurrence of each game
    unique_games = game_list.drop_duplicates(subset=['game_id'])
    
    # Create a cleaner output format
    result = pd.DataFrame({
        'Date': unique_games['Date'],
        'Home_Team': unique_games['Team'],
        'Away_Team': unique_games['Opp'],
        'Predicted_Winner': unique_games.apply(
            lambda row: row['Team'] if row['TeamPrediction'] == 'W' else row['Opp'],
            axis=1
        )
    })
    
    # Sort by date
    result = result.sort_values('Date').reset_index(drop=True)
    
    return result


In [1321]:
#get week 17 games
week17 = matches_rolling[matches_rolling['Season'] == 2024].copy(deep = True)
week17 = week17[week17['Week'] == 17]
rf_idx = 0
predictors = ['Home/AwayCode', 'TeamCode', 'OppCode']
#predictorCols = ['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd',
#'DefPassY', 'DefRushY', 'DefTO', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling',
#'OffTORolling', 'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling']
    
for predictorCols in predictorTests:
    new_predictors = predictors + predictorCols
    rf = rf_models[rf_idx]
    rf_idx += 1
    week17_preds = rf.predict(week17[new_predictors])
    week17_table = pd.DataFrame({'Predicted': week17_preds}, index=week17.index) 
    #week17_table['Predicted'] = mapWin(week17_table['Predicted'])
    week17_table = week17_table.merge(week17[['Date', 'Team', 'Opp']], left_index=True, right_index=True)
    #week17_table['Predicted'] = week17_table['Predicted'].apply(lambda x : 'W' if x==1 else 'L')
    
    #week17_table = week17_table[ week17_table['Date'] <= pd.Timestamp.now()]
    print("Predictors:", new_predictors)
    merged17 = week17_table.merge(week17_table, left_on= ['Date', 'Team'], right_on = ['Date','Opp'])

    merged17 = merged17.reindex(columns = ['Date', 'Team_x', 'Predicted_x', 'Opp_x', 'Predicted_y'])
    merged17 = merged17.rename(columns = {'Team_x':'Team', 'Predicted_x':'TeamPrediction', 'Opp_x':'Opp', 'Predicted_y':'OppPrediction'})

    
    #merged17 = combined17.merge(combined, on= ['Date', 'Team'])

    #remapping 0's and 1's to losses and wins
    mapping_cols = ['TeamPrediction', 'OppPrediction']
    for col in mapping_cols:
        merged17[col] = merged17[col].apply(lambda x : 'W' if x == 1 else 'L')

    merged17 = joinGames(merged17) #used to condense tables after analyzing the prediction set for them.

    display(merged17)

        
    #print(merged17[(merged17['Predicted'] == 'W') & (merged17['prediction_y'] == 'L')]['actual_x'].value_counts())
    print("-"*200)
    print("")

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode']


,Date,Home_Team,Away_Team,Predicted_Winner
0,2024-12-25,Baltimore Ravens,Houston Texans,Baltimore Ravens
1,2024-12-25,Kansas City Chiefs,Pittsburgh Steelers,Kansas City Chiefs
2,2024-12-26,Chicago Bears,Seattle Seahawks,Chicago Bears
3,2024-12-28,Arizona Cardinals,Los Angeles Rams,Los Angeles Rams
4,2024-12-28,Cincinnati Bengals,Denver Broncos,Cincinnati Bengals
5,2024-12-28,Los Angeles Chargers,New England Patriots,Los Angeles Chargers
6,2024-12-29,Atlanta Falcons,Washington Commanders,Atlanta Falcons
7,2024-12-29,Buffalo Bills,New York Jets,Buffalo Bills
8,2024-12-29,Carolina Panthers,Tampa Bay Buccaneers,Tampa Bay Buccaneers
9,2024-12-29,Cleveland Browns,Miami Dolphins,Cleveland Browns


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.']


,Date,Home_Team,Away_Team,Predicted_Winner
0,2024-12-25,Baltimore Ravens,Houston Texans,Baltimore Ravens
1,2024-12-25,Kansas City Chiefs,Pittsburgh Steelers,Kansas City Chiefs
2,2024-12-26,Chicago Bears,Seattle Seahawks,Seattle Seahawks
3,2024-12-28,Arizona Cardinals,Los Angeles Rams,Los Angeles Rams
4,2024-12-28,Cincinnati Bengals,Denver Broncos,Denver Broncos
5,2024-12-28,Los Angeles Chargers,New England Patriots,New England Patriots
6,2024-12-29,Atlanta Falcons,Washington Commanders,Washington Commanders
7,2024-12-29,Buffalo Bills,New York Jets,New York Jets
8,2024-12-29,Carolina Panthers,Tampa Bay Buccaneers,Tampa Bay Buccaneers
9,2024-12-29,Cleveland Browns,Miami Dolphins,Miami Dolphins


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO']


...

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling', 'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling']


,Date,Home_Team,Away_Team,Predicted_Winner
0,2024-12-25,Baltimore Ravens,Houston Texans,Baltimore Ravens
1,2024-12-25,Kansas City Chiefs,Pittsburgh Steelers,Pittsburgh Steelers
2,2024-12-26,Chicago Bears,Seattle Seahawks,Seattle Seahawks
3,2024-12-28,Arizona Cardinals,Los Angeles Rams,Los Angeles Rams
4,2024-12-28,Cincinnati Bengals,Denver Broncos,Cincinnati Bengals
5,2024-12-28,Los Angeles Chargers,New England Patriots,New England Patriots
6,2024-12-29,Atlanta Falcons,Washington Commanders,Washington Commanders
7,2024-12-29,Buffalo Bills,New York Jets,Buffalo Bills
8,2024-12-29,Carolina Panthers,Tampa Bay Buccaneers,Tampa Bay Buccaneers
9,2024-12-29,Cleveland Browns,Miami Dolphins,Cleveland Browns


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling', 'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling']


,Date,Home_Team,Away_Team,Predicted_Winner
0,2024-12-25,Baltimore Ravens,Houston Texans,Baltimore Ravens
1,2024-12-25,Kansas City Chiefs,Pittsburgh Steelers,Kansas City Chiefs
2,2024-12-26,Chicago Bears,Seattle Seahawks,Seattle Seahawks
3,2024-12-28,Arizona Cardinals,Los Angeles Rams,Arizona Cardinals
4,2024-12-28,Cincinnati Bengals,Denver Broncos,Cincinnati Bengals
5,2024-12-28,Los Angeles Chargers,New England Patriots,Los Angeles Chargers
6,2024-12-29,Atlanta Falcons,Washington Commanders,Atlanta Falcons
7,2024-12-29,Buffalo Bills,New York Jets,New York Jets
8,2024-12-29,Carolina Panthers,Tampa Bay Buccaneers,Carolina Panthers
9,2024-12-29,Cleveland Browns,Miami Dolphins,Cleveland Browns


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

